# Select, Filter, and Mutate

In this lecture, we will look at three important actions used to process data frames.  While each framework uses different names for these functions, we will use the names from the `R` library `dplyr`, namely `select`, `mutate`, and `filter`.  The most important takeaway will be that, regardless of framework or scale, we can process data frames in the same way by applying the same sequence of data verbs.

## R and Python can interact!

In [35]:
#!pip install rpy2 tzlocal
import rpy2
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [36]:
import warnings
warnings.filterwarnings('ignore')

In [37]:
%%R
rnorm(5, 2, 3)

[1] 4.678348 2.427362 4.967370 3.154490 5.622165


## We love dplyr!

In [38]:
%%R 
library(dplyr)
artists <- read.csv('./data/Artists.csv')

(artists %>%
  select(BeginDate, 
         DisplayName, 
         Nationality) %>%
  filter(BeginDate > 0) %>%
  head) -> output
output

  BeginDate     DisplayName Nationality
1      1930  Robert Arneson    American
2      1936  Doroteo Arnaiz     Spanish
3      1941     Bill Arnold    American
4      1946 Charles Arnoldi    American
5      1941     Per Arnoldi      Danish
6      1925   Danilo Aroldi     Italian


## What makes `dplyr` so great?

* Focus on data verbs
* Pipes lead to code that is
    * More readable
    * Easy to compose and debug

## Set up

Let's read in a data set in each of the three frameworks

#### `pandas` and `dfply`

In [1]:
import pandas as pd
from dfply import *
heroes = pd.read_csv('./data/heroes_information.csv')

#### `sqlalchemy` 

In [2]:
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine, func
from sqlalchemy.ext.automap import automap_base

engine = create_engine("sqlite:///databases/heroes_2_1.db")

Base = automap_base()
Base.prepare(engine, reflect=True)
Hero = Base.classes.heroes

Session = sessionmaker(bind=engine)
session = Session()

#### `pyspark`

In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, mean

spark1 = SparkSession.builder.appName('Ops').getOrCreate()
df_spark = spark1.read.csv('data/heroes_information.csv', inferSchema=True, header=True)

## Selecting Columns

The first verb, `select` 

* filters the *columns*
* At the core of `SQL` statements

## How to select

* `pandas`: pipe (`>>`) into `select`
* `sqlalchemy`: Use `session.query`
* `pyspark`: Use the `select` method

#### `select` in `pandas` + `dfply`

In [ ]:
from dfply import select as select_dfply
(heroes >>
   select_dfply(X.name, 
                X.Gender, 
                X.Weight) >>
   head
)

#### `select` expression in `sqlalchemy`

In [42]:
from sqlalchemy import select as select_sql
stmt = (select_sql([Hero.name, Hero.gender, Hero.weight]).
          select_from(Hero).
          limit(5))
print(stmt)

SELECT heroes.name, heroes.gender, heroes.weight 
FROM heroes
 LIMIT :param_1


In [44]:
pd.read_sql_query(stmt, con=engine)

,name,gender,weight
0,A-Bomb,Male,441.0
1,Abe Sapien,Male,65.0
2,Abin Sur,Male,90.0
3,Abomination,Male,441.0
4,Abraxas,Male,NaN


#### `select` in `pyspark`

In [44]:
from more_pyspark import to_pandas
pyspark_result = (df_spark.
                    select(df_spark.name, 
                           df_spark.Gender, 
                           df_spark.Weight).
                    take(5))
pyspark_result

[Row(name='A-Bomb', Gender='Male', Weight=441.0),
 Row(name='Abe Sapien', Gender='Male', Weight=65.0),
 Row(name='Abin Sur', Gender='Male', Weight=90.0),
 Row(name='Abomination', Gender='Male', Weight=441.0),
 Row(name='Abraxas', Gender='Male', Weight=-99.0)]

In [45]:
pyspark_result >> to_pandas

,Gender,Weight,name
0,Male,441.0,A-Bomb
1,Male,65.0,Abe Sapien
2,Male,90.0,Abin Sur
3,Male,441.0,Abomination
4,Male,-99.0,Abraxas


## Filtering Rows

The next verb, `filter` 

* filters the *rows*
* is related to the `SQL` `WHERE` clause

## How to filter

* `pandas`: pipe (`>>`) into `filter_by`
* `sqlalchemy`: Use `session.query.filter` or `session.query.filter_by`
* `pyspark`: Use the `where` method

#### `filter_by` in `pandas` + `dfply`

In [46]:
(heroes >>
  filter_by(X.Gender == 'Male') >>
  head)

,Id,name,Gender,Eye color,Race,Hair color,Height,Publisher,Skin color,Alignment,Weight
0,0,A-Bomb,Male,yellow,Human,No Hair,203.0,Marvel Comics,-,good,441.0
1,1,Abe Sapien,Male,blue,Icthyo Sapien,No Hair,191.0,Dark Horse Comics,blue,good,65.0
2,2,Abin Sur,Male,blue,Ungaran,No Hair,185.0,DC Comics,red,good,90.0
3,3,Abomination,Male,green,Human / Radiation,No Hair,203.0,Marvel Comics,-,bad,441.0
4,4,Abraxas,Male,blue,Cosmic Entity,Black,-99.0,Marvel Comics,-,bad,-99.0


#### `where` in a `sqlalchemy` `select` expression

In [47]:
# All SQL statements start with a select
f_stmt = (select_sql('*').
           where(Hero.gender == 'Male').
           limit(5))
print(f_stmt)

SELECT * 
FROM heroes 
WHERE heroes.gender = :gender_1
 LIMIT :param_1


In [48]:
pd.read_sql_query(f_stmt, con = engine)

,id,name,gender,eye_color,race,hair_color,height,publisher,skin_color,alignment,weight
0,0,A-Bomb,Male,yellow,Human,No Hair,203.0,Marvel Comics,None,good,441.0
1,1,Abe Sapien,Male,blue,Icthyo Sapien,No Hair,191.0,Dark Horse Comics,blue,good,65.0
2,2,Abin Sur,Male,blue,Ungaran,No Hair,185.0,DC Comics,red,good,90.0
3,3,Abomination,Male,green,Human / Radiation,No Hair,203.0,Marvel Comics,None,bad,441.0
4,4,Abraxas,Male,blue,Cosmic Entity,Black,NaN,Marvel Comics,None,bad,NaN


#### `where` in `pyspark`

In [49]:
f_result = (df_spark.
            where(df_spark.Gender == 'Male').
            take(5))
f_result >> to_pandas

,Alignment,Eye color,Gender,Hair color,Height,Id,Publisher,Race,Skin color,Weight,name
0,good,yellow,Male,No Hair,203.0,0,Marvel Comics,Human,-,441.0,A-Bomb
1,good,blue,Male,No Hair,191.0,1,Dark Horse Comics,Icthyo Sapien,blue,65.0,Abe Sapien
2,good,blue,Male,No Hair,185.0,2,DC Comics,Ungaran,red,90.0,Abin Sur
3,bad,green,Male,No Hair,203.0,3,Marvel Comics,Human / Radiation,-,441.0,Abomination
4,bad,blue,Male,Black,-99.0,4,Marvel Comics,Cosmic Entity,-,-99.0,Abraxas


## Chaining Data Verbs

* Processing df $\rightarrow$ chaining data verbs
* Accomplished through pipes/dot-chains

## Example 1 - `select` + `filter`

#### `pandas` + `dfply`

In [51]:
(heroes >>
   filter_by(X.Gender == 'Male') >>
   select_dfply(X.name, X.Gender, X.Weight) >>
   head)

,name,Gender,Weight
0,A-Bomb,Male,441.0
1,Abe Sapien,Male,65.0
2,Abin Sur,Male,90.0
3,Abomination,Male,441.0
4,Abraxas,Male,-99.0


#### `select`  expression in `sqlalchemy`

In [52]:
sel_filt_stmt = (select_sql([Hero.name, 
                             Hero.gender, 
                             Hero.weight]).
                   where(Hero.gender == 'Male').
                   limit(5))
# Excute the expression
pd.read_sql_query(sel_filt_stmt, con=engine)

,name,gender,weight
0,A-Bomb,Male,441.0
1,Abe Sapien,Male,65.0
2,Abin Sur,Male,90.0
3,Abomination,Male,441.0
4,Abraxas,Male,NaN


####  `pyspark`

In [53]:
sf_result = (df_spark.
            where(df_spark.Gender == 'Male').
            select(df_spark.name, 
                   df_spark.Gender, 
                   df_spark.Weight).
            take(5))
sf_result >> to_pandas

,Gender,Weight,name
0,Male,441.0,A-Bomb
1,Male,65.0,Abe Sapien
2,Male,90.0,Abin Sur
3,Male,441.0,Abomination
4,Male,-99.0,Abraxas


## Example 2 - `filter` + `filter`

Note that chaining `filter`s is an `and` operation.

####  `pandas` + `dfply`

In [54]:
(heroes >>
   select_dfply(X.name, X.Gender, X.Weight) >>
   filter_by(X.Gender == 'Male') >>
   filter_by(X.Weight > 0) >>
   head)

,name,Gender,Weight
0,A-Bomb,Male,441.0
1,Abe Sapien,Male,65.0
2,Abin Sur,Male,90.0
3,Abomination,Male,441.0
5,Absorbing Man,Male,122.0


#### `select`  expression in `sqlalchemy`

In [54]:
ff_stmt = (select_sql([Hero.name, 
                       Hero.gender, 
                       Hero.weight]).
            where(Hero.gender == 'Male').
            where(Hero.weight > 0).
            limit(5))
pd.read_sql_query(ff_stmt, con=engine)

,name,gender,weight
0,A-Bomb,Male,441.0
1,Abe Sapien,Male,65.0
2,Abin Sur,Male,90.0
3,Abomination,Male,441.0
4,Absorbing Man,Male,122.0


####  `pyspark`

In [55]:
ff_result = (df_spark.
               select(df_spark.name, 
                      df_spark.Gender, 
                      df_spark.Weight).
               where(df_spark.Gender == 'Male').
               where(df_spark.Weight > 0).
               take(5))
ff_result >> to_pandas

,Gender,Weight,name
0,Male,441.0,A-Bomb
1,Male,65.0,Abe Sapien
2,Male,90.0,Abin Sur
3,Male,441.0,Abomination
4,Male,122.0,Absorbing Man


## <font color="red"> Exercise 1: Blue-eyed Heroes </font>

Create a query that

1. Selects the name, Gender, and Eye Color columns
2. Filters on eye_color == 'blue'

####  `pandas` + `dfply`

#### `select`  expression in `sqlalchemy`

####  `pyspark`

## Constructing New Columns

The third verb, `mutate` 

* Creates new columns
* Changes existing columns

## How to mutate

* `pandas`: pipe (`>>`) into `mutate`
* `sqlalchemy`: Use a formula and alias in `session.query` or `select`
* `pyspark`: Use the `withColumns` method

## Example 3 - Converting Weight to kilograms

Currently, the weight column is in pounds.  Let's convert to kilograms.

####  `pandas` + `dfply`

In [57]:
(heroes >>
   select_dfply(X.name, 
                X.Gender, 
                X.Weight) >>
   mutate(Weight_kg = X.Weight/2.2046) >>
   head)

,name,Gender,Weight,Weight_kg
0,A-Bomb,Male,441.0,200.036288
1,Abe Sapien,Male,65.0,29.483807
2,Abin Sur,Male,90.0,40.823732
3,Abomination,Male,441.0,200.036288
4,Abraxas,Male,-99.0,-44.906105


#### `select`  expression in `sqlalchemy`

In [ ]:
from sqlalchemy import select
m_stmt = (select_sql([Hero.name, 
                      Hero.gender, 
                      Hero.weight, 
                      (Hero.weight/2.2046).label('Weight_kg')]).
            limit(5))
pd.read_sql_query(m_stmt, con=engine)

####  `pyspark`

In [59]:
m_result = (df_spark.
              select(df_spark.name, 
                     df_spark.Gender, 
                     df_spark.Weight).
              withColumn('Weight_kg', df_spark.Weight/2.2046).
              take(5))
m_result >> to_pandas

,Gender,Weight,Weight_kg,name
0,Male,441.0,200.036288,A-Bomb
1,Male,65.0,29.483807,Abe Sapien
2,Male,90.0,40.823732,Abin Sur
3,Male,441.0,200.036288,Abomination
4,Male,-99.0,-44.906105,Abraxas


## Referencing a new column

Each framework provides a way to reference a new column.

* `pandas` + `dfply`: Use the `X` `Intention`
* `sqlalchemy`: Use `column` function with the label from `select`
* `pyspark`: Use the `col` function with the label from `withColumn`

## Example 4 - Converting Weight to kilograms and filter

Let's find all heroes with a weight under 100kg.

####  `pandas` + `dfply`

In [60]:
(heroes >>
   select_dfply(X.name, X.Gender, X.Weight) >>
   mutate(Weight_kg = X.Weight/2.2046) >>
   filter_by(X.Weight_kg < 100) >>
   head)

,name,Gender,Weight,Weight_kg
1,Abe Sapien,Male,65.0,29.483807
2,Abin Sur,Male,90.0,40.823732
4,Abraxas,Male,-99.0,-44.906105
5,Absorbing Man,Male,122.0,55.338837
6,Adam Monroe,Male,-99.0,-44.906105


#### `select`  expression in `sqlalchemy`

In [61]:
from sqlalchemy import column
new_col_stmt = (select_sql([Hero.name, 
                            Hero.gender, 
                            Hero.weight, 
                            (Hero.weight/2.2046).label('Weight_kg')]).
                  where(column('Weight_kg') < 100).
                  limit(5))
pd.read_sql_query(new_col_stmt, con=engine)

,name,gender,weight,Weight_kg
0,Abe Sapien,Male,65.0,29.483807
1,Abin Sur,Male,90.0,40.823732
2,Absorbing Man,Male,122.0,55.338837
3,Adam Strange,Male,88.0,39.916538
4,Agent 13,Female,61.0,27.669418


####  `pyspark`

In [62]:
from pyspark.sql.functions import col
new_col_result = (df_spark.
                   select(df_spark.name, df_spark.Gender, df_spark.Weight).
                   withColumn('Weight_kg', df_spark.Weight/2.2046).
                   where(col('Weight_kg') < 100 ).
                   take(5))
new_col_result >> to_pandas

,Gender,Weight,Weight_kg,name
0,Male,65.0,29.483807,Abe Sapien
1,Male,90.0,40.823732,Abin Sur
2,Male,-99.0,-44.906105,Abraxas
3,Male,122.0,55.338837,Absorbing Man
4,Male,-99.0,-44.906105,Adam Monroe


## <font color="red"> Exercise 2: Tall Heroes </font>

Create a query that

1. Selects the name, Gender, and Height columns
2. Compute the height in inches.
    * Check [here](https://www.kaggle.com/claudiodavi/superhero-set) to determine the current units.
3. Filters on height_in > 72

####  `pandas` + `dfply`

#### `select`  expression in `sqlalchemy`

####  `pyspark`

####  `pandas` + `dfply`